# Initial Exploration of Data

The data for this exploration can be found on the opensource London Datastore:
* https://data.london.gov.uk/dataset/recorded_crime_summary
* https://data.london.gov.uk/dataset/land-area-and-population-density-ward-and-borough

## Notes on Recorded Crime Data

Data is available in two files for each level of geography - the most up to date data covering the last available 24 months only and one covering all historic full calendar years.

In March 2019, the Metropolitan Police Service started to provide offences grouped by the updated Home Office crime classifications . This currently only covers the most recent 24 months of data, but historic data using the previous categories is available separately back to January 2008.

Below is a list of the crime types covered under the new HO categories (*not available at LSOA level):

<u>Arson and Criminal Damage:</u>
* Arson
* Criminal Damage

<u>Burglary:</u>
* Burglary - Business and Community
* Burglary - Residential**

<u>Drug Offences:</u>
* Drug Trafficking
* Possession of Drugs

<u>Miscellaneous Crimes Against Society:</u> 
* Absconding from Lawful Custody 
* Bail Offences
* Bigamy 
* Concealing an Infant Death Close to Birth
* Dangerous Driving
* Disclosure, Obstruction, False or Misleading State
* Exploitation of Prostitution 
* Forgery or Use of Drug Prescription
* Fraud or Forgery Associated with Driver Records 
* Going Equipped for Stealing 
* Handling Stolen Goods 
* Making, Supplying or Possessing Articles for use i 
* Obscene Publications 
* Offender Management Act
* Other Forgery
* Other Notifiable Offences 
* Perjury 
* Perverting Course of Justice 
* Possession of False Documents 
* Profitting From or Concealing Proceeds of Crime 
* Soliciting for Prostitution
* Threat or Possession With Intent to Commit Crimina 
* Wildlife Crime

<u>Possession of Weapons:</u> 
* Other Firearm Offences 
* Possession of Firearm with Intent 
* Possession of Firearms Offences 
* Possession of Other Weapon 
* Possession of Article with Blade or Point

<u>Public Order Offences:</u> 
* Other Offences Against the State, or Public Order 
* Public Fear Alarm or Distress 
* Racially or Religiously Aggravated Public Fear 
* Violent Disorder

<u>Robbery:</u> 
* Robbery of Business Property 
* Robbery of Personal Property

<u>Sexual Offences:</u> 
* Other Sexual Offences 
* Rape

<u>Theft:</u> 
* Bicycle Theft 
* Other Theft 
* Shoplifting
* Theft from Person

<u>Vehicle Offences:</u> 
* Aggravated Vehicle Taking 
* Interfering with a Motor Vehicle 
* Theft from a Motor Vehicle 
* Theft or Taking of a Motor Vehicle

<u>Violence Against the Person:</u> 
* Homicide 
* Violence with Injury 
* Violence without Injury

 Fraud data was transferred from individual police forces to National Action Fraud in March 2013

**Prior to April 2017, police recorded burglary offence categories were split such that dwellings (domestic burglary) and buildings other than dwellings (non-domestic burglary) were separately identifiable, where:
* domestic burglary covers residential premises, including attached buildings such as garages
* non-domestic burglary covers non-residential premises, including businesses and public buildings, as well as non-attached buildings within the grounds of a dwelling, such as sheds and detached garages

**From April 2017 onwards a new classification of police recorded burglary was introduced, dividing offences into two categories of “residential” and “business and community”.
* “Residential” burglary includes all buildings or parts of buildings that are within the boundary of, or form a part of, a dwelling and includes the dwelling itself, vacant dwellings, sheds, garages, outhouses, summer houses and any other structure that meets the definition of a building. It also includes other premises used for residential purposes such as houseboats, residential care homes and hostels.
* “Business and community” burglary includes all buildings or parts of buildings that are used solely and exclusively for business purposes or are otherwise entirely outside the classification of residential burglary.

## Notes on Population Density Data

Land area and population density figures for 2001 to 2050 for wards and boroughs. Ward data begins in 2011.

Population figure included are GLA estimate (2016-based projections), 2011 Census and ONS Mid-year Estimates.

Inland area takes account of water line. Areas with large bodies of water will have a smaller inland area than total area since some of the area will be water.

Land area figures are taken from Ordnance Survey boundaries using MapInfo.

## Imports

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Fixed Variables

In [ ]:
url_borough_historical = 'https://data.london.gov.uk/download/recorded_crime_summary/934f2ddb-5804-4c6a-a17c-bdd79b33430e/MPS%20Borough%20Level%20Crime%20%28Historical%29.csv0'
url_borough_current = 'https://data.london.gov.uk/download/recorded_crime_summary/f2cf9458-24f0-47fa-bfc0-54b8d35be628/MPS%20Borough%20Level%20Crime%20%28most%20recent%2024%20months%29.csv'

## High Level Data Exploration 

### Borough Crime Data

In [ ]:
df_historical = pd.read_csv(url_borough_historical)
df_current = pd.read_csv(url_borough_current)

In [ ]:
df_historical.head()

In [ ]:
df_current.head()

Both current and historical crime datasets share the same format:
* MajorText - crime category
* MinorText - specific crime
* LookUp_BoroughName - borough
* yyyymm - year and month columns containing specific crime counts at borough levels

Historical data ends at 2021/11 with current data beginning at 2021/12 running up to the current date

In [ ]:
df_historical.shape, df_current.shape

Both datasets have a different shape

We expect column counts to be different due to the different time periods in which the data covers, however different row counts suggest some combinations of MajorText/MinorText/LookUp_BoroughName do not align in each data set

Let's see if each dataset is populated with the same values

In [ ]:
set(df_historical['MajorText']) == set(df_current['MajorText']), \
set(df_historical['MinorText']) == set(df_current['MinorText']), \
set(df_historical['LookUp_BoroughName']) == set(df_current['LookUp_BoroughName'])

Each dataset contains the same unique entries in Major/Minor/Borough columns, we I shall explore which combinations are uncommon and begin to investigate why

In [ ]:
# By concatenating and dropping duplicates I can highlight combinations that are unique to one dataset or the other
# I'm adding a prefix to the index so that I can retain knowledge of which dataset the unique combination comes from for investigative purposes

df_unique = pd.concat([df_current[['MajorText', 'MinorText', 'LookUp_BoroughName']].add_prefix('current_', axis=0),
                       df_historical[['MajorText', 'MinorText', 'LookUp_BoroughName']].add_prefix('historical_', axis=0)]).drop_duplicates(keep=False)
df_unique

There are a total of 181 combinations that are uncommon between both datasets.

I suspect that combinations may be missing in either data set due to one of two reasons:
* A particular crime having not been recorded in the time period within a borough
* A particular crime no longer being recorded beyond a certain time point

I will begin to explore the numbers below to highlight any trends, this will provide reasoning for dropping, nulling or zeroing in data where it is missing.

In [ ]:
len(df_unique['MajorText'].value_counts(ascending=True))

In [ ]:
fig, ax = plt.subplots(figsize=(10,len(df_unique['MajorText'].value_counts())/3), constrained_layout=True)
ax.set_title('Counts of Major Crimes; Unique to Hstorical or Current Data')
sns.countplot(y='MajorText', data=df_unique, order=df_unique['MajorText'].value_counts().index, width=0.6);

In [ ]:
fig, ax = plt.subplots(figsize=(10,df_unique['MinorText'].nunique()/3), constrained_layout=True)
ax.set_title('Counts of Minor Crimes; Unique to Hstorical or Current Data')
sns.countplot(y='MinorText', data=df_unique, hue='MajorText', order=df_unique['MinorText'].value_counts().index, width=0.6);

In [ ]:
fig, ax = plt.subplots(figsize=(10,len(df_unique['LookUp_BoroughName'].value_counts())/3), constrained_layout=True)
ax.set_title('Crime Counts by Borough; Unique to Hstorical or Current Data')
df_unique['LookUp_BoroughName'].value_counts(ascending=True).plot(kind='barh', width=0.6);

The above counts highlight a few things:
* Miscellaneous crime are most commonly absent - however this is a large catergory filled with more obscure crimes so it not a surprise to see it rank highly
* Minor category crimes link one-to-one with boroughs - Historical Fraud and Forgery and Wildlife Crime rank particularly high here and will be investigated
* I will investigate crime levels in Boroughs that rank highly on this list

In [ ]:
df_temp = df_historical.loc[[int(x.split('_')[-1]) for x in df_unique.filter(like='historical', axis=0).index]].groupby('MinorText').sum(numeric_only=True).reset_index()

fig, axes = plt.subplots(nrows=int(np.ceil(len(df_temp)/2)), ncols=2, figsize=(10,len(df_temp) + np.mod(len(df_temp), 2)), constrained_layout=True)
fig.suptitle('Crime Trends Unique to Historical Data', fontsize=20)

if np.mod(len(df_temp), 2):
    axes[-1, -1].set_axis_off()

for index, row in df_temp.iterrows():

    ax=axes[int(np.floor(index/2)), np.mod(index, 2)]
    row[1:].plot(ax=ax, title=row.iloc[0])

Most crimes in the above figure show a sporadic trend, and so it is conceivable that they have simply not occured within the current dataset.

A few crimes show a spike at some point in the historical data set: Bail Offences, Soliciting for Prostitution, Violent Disorder & Wildlife Crime. However, they show a more sporadic trend in the most recent months, and so again, it is conceivable that they have simply not occured within the current dataset. It may be worth investigating the reasoning for these spikes and drop-offs, such as changes in recording of crime, political campaigns and/or any socio-economic changes, however I will leave such in-depth investigation out of scope for this project.

There is one chart that stands out in the above figure, and that is Historical Fraud and Forgery. After initially recording rather high numbers of crime, there is a very sudden drop off, I will investigate further

In [ ]:
df_temp = df_current.loc[[int(x.split('_')[-1]) for x in df_unique.filter(like='current', axis=0).index]].groupby('MinorText').sum(numeric_only=True).reset_index()

fig, axes = plt.subplots(nrows=int(np.ceil(len(df_temp)/2)), ncols=2, figsize=(10,len(df_temp) + np.mod(len(df_temp), 2)), constrained_layout=True)
fig.suptitle('Crime Trends Unique to Current Data', fontsize=20)

if np.mod(len(df_temp), 2):
    axes[-1, -1].set_axis_off()

for index, row in df_temp.iterrows():

    ax=axes[int(np.floor(index/2)), np.mod(index, 2)]
    row[1:].plot(ax=ax, title=row.iloc[0])

The crimes shown above are unique to the current dataset, and they all show a very sporadic trend in recorded crime. I will assume that they are not included in the historical dataset due to low level recording of these crimes.

In [ ]:
df_temp = df_historical.loc[[int(x.split('_')[-1]) for x in df_unique.filter(like='historical', axis=0).index]].groupby('LookUp_BoroughName').sum(numeric_only=True).reset_index()

fig, axes = plt.subplots(nrows=int(np.ceil(len(df_temp)/2)), ncols=2, figsize=(10,len(df_temp) + np.mod(len(df_temp), 2)), constrained_layout=True)
fig.suptitle('Crime Trends by Borough; Unique to Historical Data', fontsize=20)

if np.mod(len(df_temp), 2):
    axes[-1, -1].set_axis_off()

for index, row in df_temp.iterrows():

    ax=axes[int(np.floor(index/2)), np.mod(index, 2)]
    row[1:].plot(ax=ax, title=row.iloc[0])

The above charts appear to be heavily influenced by the trend in recorded Historical Fraud and Forgery

In [ ]:
df_temp = df_current.loc[[int(x.split('_')[-1]) for x in df_unique.filter(like='current', axis=0).index]].groupby('LookUp_BoroughName').sum(numeric_only=True).reset_index()

fig, axes = plt.subplots(nrows=int(np.ceil(len(df_temp)/2)), ncols=2, figsize=(10, len(df_temp) + np.mod(len(df_temp), 2)), constrained_layout=True)
fig.suptitle('Crime Trends by Borough; Unique to Historical Data', fontsize=20)

if np.mod(len(df_temp), 2):
    axes[-1, -1].set_axis_off()

for index, row in df_temp.iterrows():

    ax=axes[int(np.floor(index/2)), np.mod(index, 2)]
    row[1:].plot(ax=ax, title=row.iloc[0])

The crimes shown above are unique to the current dataset, and they all show a very sporadic trend in recorded crime. I will assume that they are not included in the historical dataset due to low level recording of these crimes.

#### Historical Fraud and Forgery

In [ ]:
df_unique[df_unique['MinorText']=='Historical Fraud and Forgery']

In [ ]:
df_current[df_current['MinorText']=='Historical Fraud and Forgery']

Historical Fraud and Forgery appears overwhelmingly in the historical data, and only once in the current dataset. 

Noteably the one occurance appears in 2021/12 which is the earliest point in the dataset.

In [ ]:
df_historical[df_historical['MinorText']=='Historical Fraud and Forgery'].sum()[3:].plot()

In [ ]:
df_historical[df_historical['MinorText']=='Historical Fraud and Forgery'].sum()['201112':'201308'].plot(kind='bar')

The above plots show a sudden drop off in recorded Historical Fraud and Forgery around 2013/02

Taken from the dataset notes:
* Fraud data was transferred from individual police forces to National Action Fraud in March 2013

Due to a short timeline of data that is no longer relevent, I will make the decision to drop Historical Fraud and Forgery from both historical and current datasets

In [ ]:
df_historical.drop(df_historical[df_historical['MinorText']=='Historical Fraud and Forgery'].index, inplace=True)
df_current.drop(df_historical[df_historical['MinorText']=='Historical Fraud and Forgery'].index, inplace=True)

In [ ]:
# By concatenating and dropping duplicates I can highlight combinations that are unique to one dataset or the other
# I'm adding a prefix to the index so that I can retain knowledge of which dataset the unique combination comes from for investigative purposes

df_unique = pd.concat([df_current[['MajorText', 'MinorText', 'LookUp_BoroughName']].add_prefix('current_', axis=0),
                       df_historical[['MajorText', 'MinorText', 'LookUp_BoroughName']].add_prefix('historical_', axis=0)]).drop_duplicates(keep=False)
df_unique

In [ ]:
df_temp = df_historical.loc[[int(x.split('_')[-1]) for x in df_unique.filter(like='historical', axis=0).index]].groupby('LookUp_BoroughName').sum(numeric_only=True).reset_index()

fig, axes = plt.subplots(nrows=int(np.ceil(len(df_temp)/2)), ncols=2, figsize=(10,len(df_temp) + np.mod(len(df_temp), 2)), constrained_layout=True)
fig.suptitle('Crime Trends by Borough; Unique to Historical Data', fontsize=20)

if np.mod(len(df_temp), 2):
    axes[-1, -1].set_axis_off()

for index, row in df_temp.iterrows():

    ax=axes[int(np.floor(index/2)), np.mod(index, 2)]
    row[1:].plot(ax=ax, title=row.iloc[0])

In [ ]:
df_current.isna().sum().any(), df_historical.isna().sum().any()

In [ ]:
df_historical.head()

In [ ]:
df_historical.shape, df_current.shape

In [ ]:
pd.concat([df_historical, df_current], ignore_index=True, keys=('MajorText', 'MinorText', 'LookUp_BoroughName'), axis=1)

In [ ]:
df_total = df_historical.merge(df_current, how='outer', on=('MajorText', 'MinorText', 'LookUp_BoroughName')).fillna(0)
df_total

In [ ]:
1713*167

In [ ]:
df_total.head()

In [ ]:
df_reshaped = df_total.set_index(['MajorText', 'MinorText', 'LookUp_BoroughName']).stack().reset_index().copy()
df_reshaped.head()

In [ ]:
df_reshaped.dtypes

In [ ]:
from datetime import datetime

In [ ]:
datetime.strptime('201004', '%Y%m')

In [ ]:
df_reshaped['level_3']

In [ ]:
df_reshaped['level_3'] = pd.to_datetime(df_reshaped['level_3'], format='%Y%m')
df_reshaped

In [ ]:
df_reshaped['level_3'].value_counts()

In [ ]:
df_reshaped.set_index('level_3', inplace=True)
df_reshaped